# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
!git clone https://github.com/jantic/DeOldify.git

fatal: destination path 'DeOldify' already exists and is not an empty directory.


In [2]:
cd DeOldify

C:\Users\xndc4\PycharmProjects\masters_project\DeOldify


In [3]:
!pip install -r requirements-colab.txt

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118


In [5]:
import torch
print(torch.cuda.is_available())

True


In [6]:
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

<DeviceId.GPU0: 0>

In [7]:
from fastai.callbacks.tensorboard import *
from fastai.vision.gan import *
from deoldify.generators import *
from deoldify.critics import *
from deoldify.dataset import *
from deoldify.loss import *
from deoldify.save import *
from PIL import Image

Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [8]:
from deoldify.visualize import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

In [9]:
import kagglehub
import os
import shutil
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from PIL import Image, ImageDraw

path = kagglehub.dataset_download("akash2sharma/tiny-imagenet")
train_ds = './data/custom_dataset/norm/train/'
sample_train = './data/custom_dataset/norm/sample_train/'
valid_ds = './data/custom_dataset/norm/val/'
test_ds = './data/custom_dataset/norm/test/'
train_bandw = './data/custom_dataset/bandw/train/'
val_bandw = './data/custom_dataset/bandw/val/'
test_bandw = './data/custom_dataset/bandw/test/'

os.makedirs(train_ds, exist_ok=True)
os.makedirs(sample_train, exist_ok=True)
os.makedirs(valid_ds, exist_ok=True)
os.makedirs(test_ds, exist_ok=True)
os.makedirs(train_bandw, exist_ok=True)
os.makedirs(val_bandw, exist_ok=True)
os.makedirs(test_bandw, exist_ok=True)

In [24]:
all_files = []
full_train = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.lower().endswith((".png", ".jpg", ".jpeg")):
            all_files.append(os.path.join(root, file.lower()))

def copy_file(file_path):
    if 'val_' in file_path:
        shutil.copy(file_path, valid_ds)
    elif 'test_' in file_path:
        shutil.copy(file_path, test_ds)
    else:
        shutil.copy(file_path, train_ds)
        full_train.append(file_path)

# Use a thread pool to parallelize copying
with ThreadPoolExecutor() as executor:
    executor.map(copy_file, all_files)
    list(tqdm(executor.map(copy_file, all_files), total=len(all_files)))


100%|██████████| 240000/240000 [02:40<00:00, 1491.88it/s]


In [32]:
# path = Path('./data/custom_dataset')
# path = Path('data/imagenet/ILSVRC/Data/CLS-LOC')
path_hr = sample_train
path_lr = train_bandw
pretrained_gen_path = 'ColorizeArtistic_gen'

proj_id = 'ArtisticModel'

gen_name = proj_id + '_gen'
pre_gen_name = gen_name + '_0'
crit_name = proj_id + '_crit'

name_gen = proj_id + '_image_gen'
path_gen = './data/custom_dataset/name_gen'

TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

nf_factor = 1.5
pct_start = 1e-8

In [33]:
def get_data(bs:int, sz:int, keep_pct:float):
    return get_colorize_data(sz=sz, bs=bs, crappy_path=path_lr, good_path=path_hr,
                             random_seed=None, keep_pct=keep_pct)

def get_crit_data(classes, bs, sz):
    src = ImageList.from_folder(path, include=classes, recurse=True).split_by_rand_pct(0.1, seed=42)
    ll = src.label_from_folder(classes=classes)
    data = (ll.transform(get_transforms(max_zoom=2.), size=sz)
           .databunch(bs=bs).normalize(imagenet_stats))
    return data

def save_preds(dl):
    i=0
    names = dl.dataset.items

    for b in dl:
        preds = learn_gen.pred_batch(batch=b, reconstruct=True)
        for o in preds:
            o.save(path_gen/names[i].name)
            i += 1

def save_gen_images():
    if path_gen.exists(): shutil.rmtree(path_gen)
    path_gen.mkdir(exist_ok=True)
    data_gen = get_data(bs=bs, sz=sz, keep_pct=0.085)
    save_preds(data_gen.fix_dl)
    PIL.Image.open(path_gen.ls()[0])

In [34]:
import cv2
import numpy as np
import random
from pathlib import Path  # Also needed

# Save Image

def save_image(filename, image):
    # Extract filename
    output_dir = Path(path_lr)
    output_dir.mkdir(parents=True, exist_ok=True)
    dest = output_dir / filename
    image.save(dest, format="JPEG")

def save_cv2(filename, image):
    output_dir = Path(path_lr)
    output_dir.mkdir(parents=True, exist_ok=True)
    dest = output_dir / filename
    cv2.imwrite(str(dest), image)

def save_image_before(filename, image):
    # Extract filename
    output_dir = Path(path_hr)
    output_dir.mkdir(parents=True, exist_ok=True)
    dest = output_dir / filename
    image.save(dest, format="JPEG")

def save_cv2_before(filename, image):
    output_dir = Path(path_hr)
    output_dir.mkdir(parents=True, exist_ok=True)
    dest = output_dir / filename
    cv2.imwrite(str(dest), image)

# Rotation – Turning the image around a point.


def apply_image_rotation(image_path):
    image = Image.open(image_path)
    image_name = Path(image_path).stem
    image_ext = ".jpeg"


    for i in range(90, 271, 90):
        # Rotate the image (degrees counter-clockwise)
        rotated_image = image.rotate(i, expand=True)  # Use expand=True to resize the output to fit the entire image
        filename = f"{image_name}_rotation_{i}{image_ext}"
        save_image_before(filename, image)
        save_image(filename, rotated_image)

# Gaussian blur – Smooths the image using a Gaussian function.


def apply_gaussian_blur(image_path):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    image_ext = ".jpeg"
    sigmaX_range = np.arange(0.3, 5.1, 0.3)
    for sigmaX in sigmaX_range:
        blurred = cv2.GaussianBlur(image, (15, 15), sigmaX)
        filename = f"{image_name}_gaussian_blur_{sigmaX:.2f}{image_ext}"
        save_cv2_before(filename, image)
        save_cv2(filename, blurred)

# Motion blur – Simulates movement during image capture.


def apply_motion_blur(image_path):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    image_ext = ".jpeg"
    kernel_sizes = np.arange(5, 46, 1)
    angles = np.arange(0, 181, 45)
    for kernels in kernel_sizes:
        for angle in angles:
            # Create the motion_blur blur kernel
            kernel = np.zeros((kernels, kernels))
            kernel[int((kernels - 1) / 2), :] = np.ones(kernels)

            # Rotate the kernel to the desired angle
            center = (float(kernels // 2), float(kernels // 2))
            rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
            kernel = cv2.warpAffine(kernel, rot_mat, (kernels, kernels))

            # Normalize the kernel
            kernel = kernel / np.sum(kernel)

            # Apply the kernel to the image
            blurred = cv2.filter2D(image, -1, kernel)
            filename = f"{image_name}_motion_blur_{int(kernels)}{int(angle)}{image_ext}"
            save_cv2_before(filename, image)
            save_cv2(filename, blurred)

# Defocus blur – Mimics an out-of-focus lens effect.


def apply_defocus_blur(image_path):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    image_ext = ".jpeg"
    radii = [i for i in range(3, 21, 1)]
    for radius in radii:
        kernel_size = radius * 2 + 1
        y, x = np.ogrid[:kernel_size, :kernel_size]
        center = (radius, radius)
        mask = (x - center[0]) ** 2 + (y - center[1]) ** 2 <= radius ** 2
        kernel = np.zeros((kernel_size, kernel_size), dtype=np.float32)
        kernel[mask] = 1
        kernel /= np.sum(kernel)

        # Apply the kernel using filter2D
        blurred = cv2.filter2D(image, -1, kernel)
        filename = f"{image_name}_defocus_blur_{int(radius)}{image_ext}"
        save_cv2_before(filename, image)
        save_cv2(filename, blurred)

# Radial blur – Blur increases from center outward (like a zoom effect).


def apply_radial_blur(image_path):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    image_ext = ".jpeg"
    strengths = [i for i in range(5, 31, 1)]
    for strength in strengths:
        h, w = image.shape[:2]
        center = (w // 2, h // 2)
        blurred = np.zeros_like(image, dtype=np.float32)

        # Blend progressively zoomed images
        for i in range(1, strength + 1):
            zoom_factor = 1 + (i / (strength * 10))
            M = cv2.getRotationMatrix2D(center, 0, zoom_factor)
            zoomed = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR)
            alpha = 1.0 / strength
            blurred += zoomed.astype(np.float32) * alpha

        radial_blur = np.clip(blurred, 0, 255).astype(np.uint8)
        filename = f"{image_name}_radial_blur_{int(strength)}{image_ext}"
        save_cv2_before(filename, image)
        save_cv2(filename, radial_blur)

# Contrast change – Altering difference between light/dark areas.


def apply_adaptive_contrast_clahe(image_path):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    image_ext = ".jpeg"
    clip_limits = [i for i in range(1, 12, 1)]
    tile_grid_sizes = [(i, i) for i in range(4, 16, 1)]
    for clip_limit in clip_limits:
        for tile_grid_size in tile_grid_sizes:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
            enhanced = clahe.apply(gray)
            blurred = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)
            filename = f"{image_name}_contrast_clahe_{int(clip_limit)}{int(tile_grid_size[0])}{image_ext}"
            save_cv2_before(filename, image)
            save_cv2(filename, blurred)

# Color jittering – Random small changes to brightness, contrast, etc.


def apply_color_jitter(image_path):
    image = cv2.imread(image_path)
    image_name = Path(image_path).stem
    image_ext = ".jpeg"
    brightness_arr = np.arange(0.1, 1.0, 0.1)
    contrast_arr = np.arange(0.1, 1.0, 0.1)
    saturation_arr = np.arange(0.1, 1.0, 0.1)
    hue_arr = [i for i in range(0, 181, 10)]

    for brightness in brightness_arr:
        for contrast in contrast_arr:
            for saturation in saturation_arr:
                for hue in hue_arr:

                    if brightness == 0.1 and contrast == 0.2 and saturation == 0.6 and hue == 130: return
                    # Randomly adjust brightness
                    brightness_factor = random.uniform(1 - brightness, 1 + brightness)
                    image = np.clip(image * brightness_factor, 0, 255).astype(np.uint8)

                    # Convert to HSV for color adjustments
                    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

                    # Randomly adjust saturation
                    saturation_factor = random.uniform(1 - saturation, 1 + saturation)
                    hsv[..., 1] = np.clip(hsv[..., 1] * saturation_factor, 0, 255)

                    # Randomly adjust hue
                    hue_shift = random.randint(-hue, hue)
                    h = hsv[..., 0].astype(int)
                    h = (h + hue_shift) % 180
                    hsv[..., 0] = h.astype(np.uint8)  # Ensure hue is in [0, 180]

                    # Convert back to BGR
                    image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

                    # Randomly adjust contrast by scaling pixel values
                    contrast_factor = random.uniform(1 - contrast, 1 + contrast)
                    new_img = np.clip((image - 128) * contrast_factor + 128, 0, 255).astype(np.uint8)
                    filename = f"{image_name}_color_jitter_{brightness:.2f}{contrast:.2f}{saturation:.2f}{int(hue)}{image_ext}"
                    save_cv2_before(filename, image)
                    save_cv2(filename, new_img)

# Gaussian noise – Random pixel values with normal distribution.


def apply_gaussian_noise(image_path, mean=0):
    """Add Gaussian noise to an image."""
    # Open the image and convert it to a numpy array
    image = Image.open(image_path)
    image_name = Path(image_path).stem
    image_ext = ".jpeg"
    image_array = np.array(image)
    sigmas = [i for i in range(10, 50, 5)]
    for sigma in sigmas:
        # Generate Gaussian noise
        gauss_noise = np.random.normal(mean, sigma, image_array.shape)

        # Add the noise to the image array
        noisy_image_array = image_array + gauss_noise

        # Clip the values to be within valid range (0 to 255)
        noisy_image_array = np.clip(noisy_image_array, 0, 255)

        # Convert the noisy image array back to an image
        noisy_image = Image.fromarray(np.uint8(noisy_image_array))
        filename = f"{image_name}_gaussian_noise_{int(sigma)}{image_ext}"
        save_image_before(filename, image)
        save_image(filename, noisy_image)

# Salt & pepper noise – Random black and white pixels.


def apply_salt_and_pepper_noise(image_path):
    # Open the image and convert it to a numpy array
    image = Image.open(image_path)
    image_name = Path(image_path).stem
    image_ext = ".jpeg"
    image_array = np.array(image)

    salt = np.arange(0.01, 0.1, 0.01)
    pepper = np.arange(0.01, 0.1, 0.01)
    for salt_prob in salt:
        for pepper_prob in pepper:
            # Generate random salt-and-pepper noise
            noisy_image_array = image_array.copy()

            if noisy_image_array.ndim != 3: continue
            # Add salt (white pixels)
            num_salt = int(salt_prob * image_array.size)
            salt_coords = [np.random.randint(0, i, num_salt) for i in image_array.shape[:2]]
            noisy_image_array[salt_coords[0], salt_coords[1], :] = 255

            # Add pepper (black pixels)
            num_pepper = int(pepper_prob * image_array.size)
            pepper_coords = [np.random.randint(0, i, num_pepper) for i in image_array.shape[:2]]
            noisy_image_array[pepper_coords[0], pepper_coords[1], :] = 0

            # Convert the noisy image array back to an image
            noisy_image = Image.fromarray(np.uint8(noisy_image_array))
            filename = f"{image_name}_salt_pepper_{salt_prob:.2f}{pepper_prob:.2f}{image_ext}"
            save_image_before(filename, image)
            save_image(filename, noisy_image)

# Speckle noise – Multiplicative noise, often in radar/medical images.


def apply_speckle_noise(image_path):
    sigmas = np.arange(0.05, 0.51, 0.05)

    # Open the image and convert it to a numpy array
    image = Image.open(image_path)
    image_name = Path(image_path).stem
    image_ext = ".jpeg"
    image_array = np.array(image)

    for sigma in sigmas:
        # Generate speckle noise (multiplicative noise)
        noise = np.random.normal(0, sigma, image_array.shape)

        # Add speckle noise to the image
        noisy_image_array = image_array + image_array * noise

        # Clip the values to be within valid range (0 to 255)
        noisy_image_array = np.clip(noisy_image_array, 0, 255)

        # Convert the noisy image array back to an image
        noisy_image = Image.fromarray(np.uint8(noisy_image_array))
        filename = f"{image_name}_speckle_noise_{sigma:.2f}{image_ext}"
        save_image_before(filename, image)
        save_image(filename, noisy_image)

# Banding artifacts – Horizontal or vertical lines due to poor encoding.


def apply_banding_artifacts(image_path):
    colors = [2, 16, 64]
    image = Image.open(image_path)
    image_name = Path(image_path).stem
    image_ext = ".jpeg"
    image_array = np.array(image)

    # Normalize the image to the range [0, 1]
    image_array = image_array / 255.0

    for num_colors in colors:
        # Quantize each color channel to a limited number of levels
        quantized_image_array = np.floor(image_array * num_colors) / num_colors

        # Convert the image array back to the range [0, 255]
        quantized_image_array = (quantized_image_array * 255).astype(np.uint8)

        # Convert the noisy image array back to an image
        noisy_image = Image.fromarray(quantized_image_array)
        filename = f"{image_name}_artifacts_{int(num_colors)}{image_ext}"
        save_image_before(filename, image)
        save_image(filename, noisy_image)

# Occlusion – Random parts of image covered (e.g., with boxes).


def apply_random_occlusion_boxes(image_path):
    for i in range(10):
        # Create a drawing object to draw on the image
        before_image = Image.open(image_path)
        after_image = before_image.copy()
        image_name = Path(image_path).stem
        image_ext = ".jpeg"
        draw = ImageDraw.Draw(after_image)

        # Get image dimensions
        width, height = after_image.size

        for j in range(10):
            # Randomly select the size of the box within the maximum size
            box_width = np.random.randint(int(width * 0.125), int(width * 0.3))
            box_height = np.random.randint(int(width * 0.125), int(width * 0.3))

            # Randomly select top-left corner for the box (ensuring it fits within the image)
            top_left_x = np.random.randint(0, width - box_width)
            top_left_y = np.random.randint(0, height - box_height)

            # Define the box coordinates (top-left and bottom-right corners)
            bottom_right_x = top_left_x + box_width
            bottom_right_y = top_left_y + box_height

            # Draw a black box (occlusion)
            draw.rectangle([top_left_x, top_left_y, bottom_right_x, bottom_right_y], fill="black")
            filename = f"{image_name}_boxes_{int(i)}{int(j)}{image_ext}"
            save_image_before(filename, before_image)
            save_image(filename, after_image)


def create_training_images(fn,i):
    apply_image_rotation(fn)
    apply_gaussian_blur(fn)
    apply_adaptive_contrast_clahe(fn)
    apply_color_jitter(fn)
    apply_gaussian_noise(fn)
    apply_salt_and_pepper_noise(fn)
    apply_speckle_noise(fn)
    apply_banding_artifacts(fn)
    apply_random_occlusion_boxes(fn)


In [35]:
il = full_train[:10] #100000 train, 240000 total
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(create_training_images, str(fn), i) for i, fn in enumerate(il)]
    for f in futures:
        f.result()

In [36]:
import os

path_hr = './data/custom_dataset/norm/sample_train/'
files = os.listdir(path_hr)
files2 = os.listdir(path_lr)
print(len(files))
print(len(files2))

6300
6300


In [38]:
bs=88
sz=64
keep_pct=1.0
data_gen = get_data(bs=bs, sz=sz, keep_pct=keep_pct)
learn_gen = gen_learner_deep(data=data_gen, gen_loss=FeatureLoss(), nf_factor=nf_factor)
learn_gen.load(pretrained_gen_path, with_opt=False)
learn_gen.callback_fns.append(partial(ImageGenTensorboardWriter, base_dir=TENSORBOARD_PATH, name='GenPre'))
learn_gen.fit_one_cycle(1, pct_start=0.8, max_lr=slice(1e-3))
learn_gen.save(pre_gen_name)
learn_gen.unfreeze()
learn_gen.fit_one_cycle(1, pct_start=pct_start,  max_lr=slice(3e-7, 3e-4))
learn_gen.save(pre_gen_name)


C:\Users\xndc4\AppData\Roaming\Python\Python311\site-packages\torch\nn\functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
C:\Users\xndc4\AppData\Roaming\Python\Python311\site-packages\torch\nn\functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
C:\Users\xndc4\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
C:\Users\xndc4\AppData\Roaming\Python\Python311\site-packages\torchvisi

epoch,train_loss,valid_loss,time
0,6.262414,5.705962,12:58


Summary name /metrics/train_loss is illegal; using metrics/train_loss instead.
Summary name /metrics/train_loss is illegal; using metrics/train_loss instead.
Summary name /metrics/valid_loss is illegal; using metrics/valid_loss instead.


epoch,train_loss,valid_loss,time
0,5.394530,5.327067,12:12


Summary name /metrics/train_loss is illegal; using metrics/train_loss instead.
Summary name /metrics/train_loss is illegal; using metrics/train_loss instead.
Summary name /metrics/valid_loss is illegal; using metrics/valid_loss instead.
